In [ ]:
snakemake.params["scenario"]["import_buffer"] = "annually"

In [ ]:
import logging
import pickle
import re

import dateutil
import numpy as np
import pandas as pd
import pypsa
from _helpers import configure_logging

logger = logging.getLogger(__name__)

if __name__ == "__main__":

    scenario = snakemake.params["scenario"]
    configure_logging(snakemake)

    with open(snakemake.input["additional_components"], "rb") as f:
        override_component_attrs = pickle.load(f)

    network = pypsa.Network(override_component_attrs=override_component_attrs)
    network.import_from_netcdf(snakemake.input["network"])

    # Determine all loads related to imports (on the importer side of the ESC)
    import_loads = network.loads[network.loads.bus.str.contains("imp")]

    # Read load profile from file
    df = pd.read_csv(snakemake.input["import_profile"])

    # File format consistency checks
    if {"snapshot", "profile [p.u.]"} != set(
        pd.read_csv(snakemake.input["import_profile"]).columns
    ):
        logger.error(
            f"Import profile file '{snakemake.input['import_profile']}' must have "
            f"exactly two columns 'snapshot' and 'profile [p.u.]'"
        )
    elif np.any(pd.to_datetime(df["snapshot"]) != network.snapshots):
        logger.error(
            f"At least one 'snapshot' entry in '{snakemake.input['import_profile']}' "
            f"does not match the network snapshots: {network.snapshots[:5], ...}."
        )

    df = df.set_index("snapshot")

    # Apply the import_profile to all importing loads in the network
    for load, load_s in import_loads.iterrows():
        import_profile = load_s["p_set"] * df["profile [p.u.]"]
        network.loads_t["p_set"][load] = import_profile
        network.loads.loc[load, "p_set"] = 0

    # Add optional buffers to the import loads
    if scenario["import_buffer"] is False:
        logger.info("Imports are not buffered.")
    elif scenario["import_buffer"] == "annually":
        logger.info("Imports are buffered on an annual basis, adding buffer stores.")

        # Implement annual import_buffer by adding a store
        # to the bus of each load on the import side
        network.madd(
            "Store",
            names="Buffer: " + import_loads.index,
            bus=import_loads.bus.values,
            e_nom_extendable=True,
            e_cyclic=True,
        )

    # For informative purposes in any configuration case
    logger.info(f"\t import_buffer:{scenario['import_buffer']}.")

    network.export_to_netcdf(snakemake.output["network"])